In [46]:
#!pip install delta-spark==1.2.1
#!pyspark --version

In [47]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .master('local[*]')
    .appName('Quickstart DeltaLake')
    .config('spark.jars.packages', 'io.delta:delta-core_2.12:1.2.1')
    .config('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')
    .config('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog')
    .getOrCreate()
)

In [48]:
sc = spark.sparkContext
sc.setLogLevel('ERROR')

In [49]:
df = spark.read.format('json').load('../data/')

In [50]:
columns = ['user_id', 'email']
df = df.select(columns)
df.columns

['user_id', 'email']

In [51]:
df.show(n=1, vertical=True, truncate=False)

-RECORD 0-------------------------
 user_id | 1703                   
 email   | daron.bailey@email.com 
only showing top 1 row



In [52]:
if df.rdd.getNumPartitions() > 3:
    df.coalesce(1).write.format('delta').mode('overwrite').save('../delta/bronze/user')
else:
    df.write.format('delta').mode('overwrite').save('../delta/bronze/user')

In [53]:
from delta.tables import DeltaTable

DeltaTable.forPath(spark, '../delta/bronze/user').history(1).show(vertical=True, truncate=False)

_ = DeltaTable.forPath(spark, '../delta/bronze/user').toDF()

-RECORD 0-----------------------------------------------------------------------------
 version             | 0                                                              
 timestamp           | 2022-07-11 13:29:02.304                                        
 userId              | null                                                           
 userName            | null                                                           
 operation           | WRITE                                                          
 operationParameters | {mode -> Overwrite, partitionBy -> []}                         
 job                 | null                                                           
 notebook            | null                                                           
 clusterId           | null                                                           
 readVersion         | null                                                           
 isolationLevel      | Serializable        

In [54]:
ids = '1703, 3650'

In [55]:
_.where(f'user_id in ({ids})').show()

+-------+--------------------+
|user_id|               email|
+-------+--------------------+
|   1703|daron.bailey@emai...|
|   3650|jonah.barrows@ema...|
+-------+--------------------+



In [56]:
delta_object = DeltaTable.forPath(spark, '../delta/bronze/user')
delta_object

In [57]:
# before delete
_.where('user_id == 1703').show(truncate=False)
_.count()

+-------+----------------------+
|user_id|email                 |
+-------+----------------------+
|1703   |daron.bailey@email.com|
+-------+----------------------+



600

In [58]:
df.select('user_id').distinct().show()

+-------+
|user_id|
+-------+
|   5385|
|    720|
|   8887|
|   9952|
|   7032|
|   4161|
|   2364|
|   5099|
|   2961|
|   6762|
|   3909|
|    343|
|   9000|
|   9045|
|   8222|
|    402|
|   1312|
|   4746|
|   3650|
|   1805|
+-------+
only showing top 20 rows



In [59]:
ids_delete = '5385, 5099, 402'
df.where(f'user_id in ({ids_delete})').show()


+-------+--------------------+
|user_id|               email|
+-------+--------------------+
|   5099|chang.senger@emai...|
|    402|lynetta.zulauf@em...|
|   5385|winston.kirlin@em...|
+-------+--------------------+



In [60]:
# after delete
_.count()

600

In [61]:
_.where('user_id == 1805').show()

+-------+--------------------+
|user_id|               email|
+-------+--------------------+
|   1805|renee.hahn@email.com|
+-------+--------------------+



In [62]:
# update
delta_object.update(
    condition='email = "renee.hahn@email.com"',
    set = { 'email': '"jack.sparrow@pirates.com"'}
)

In [63]:
_.where('user_id == 1805').show(truncate=False)

+-------+------------------------+
|user_id|email                   |
+-------+------------------------+
|1805   |jack.sparrow@pirates.com|
+-------+------------------------+



In [64]:
_.show()

+-------+--------------------+
|user_id|               email|
+-------+--------------------+
|   1703|daron.bailey@emai...|
|   3650|jonah.barrows@ema...|
|   8809|carla.hansen@emai...|
|   4606|tomas.ledner@emai...|
|      1|alyse.ortiz@email...|
|   9245|russell.kulas@ema...|
|   3425|armida.lehner@ema...|
|   4264|tad.sanford@email...|
|   1668|rosia.jones@email...|
|    343|candy.conroy@emai...|
|   7393|dulcie.gottlieb@e...|
|   3909|rodrigo.reynolds@...|
|   9952|jenna.bode@email.com|
|   2364|dan.herman@email.com|
|   1611|stanley.witting@e...|
|   1723|clarinda.kilback@...|
|   7032|charley.carroll@e...|
|    549|cameron.harris@em...|
|   4161|reyes.stracke@ema...|
|    503|jolynn.schulist@e...|
+-------+--------------------+
only showing top 20 rows



In [65]:
values = [(1703, 'sheldon.cooper@thebigbang.com'), (3650, 'rajesh.koothrappali@thebigbang.com')] 
cols   = ['user_id', 'email']

In [66]:
values, cols

([(1703, 'sheldon.cooper@thebigbang.com'),
  (3650, 'rajesh.koothrappali@thebigbang.com')],
 ['user_id', 'email'])

In [67]:
new_data = spark.createDataFrame(values, cols)
new_data.show(truncate=False)

+-------+----------------------------------+
|user_id|email                             |
+-------+----------------------------------+
|1703   |sheldon.cooper@thebigbang.com     |
|3650   |rajesh.koothrappali@thebigbang.com|
+-------+----------------------------------+



In [68]:
# upserts
(
    delta_object.alias('old_data')
        .merge(
            new_data.alias('new_data'), condition = 'old_data.user_id = new_data.user_id'
        )
        .whenMatchedUpdateAll(
            condition = 'old_data.user_id = new_data.user_id'
        )
        .whenNotMatchedInsertAll()
    .execute()
)

In [69]:
ids_update = '1703, 3650'
_.where(f'user_id in ({ids_update})').show(truncate=False)

+-------+----------------------------------+
|user_id|email                             |
+-------+----------------------------------+
|1703   |sheldon.cooper@thebigbang.com     |
|3650   |rajesh.koothrappali@thebigbang.com|
+-------+----------------------------------+



In [70]:
_.count()

600

In [71]:
delta_object.history().show(truncate=False, vertical=True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 2                                                                                                                                                                                                                                                                                  
 timestamp           | 2022-07-11 13:29:16.574                                                                                                                                                                                                                                                            
 userId              | null                                                                            

In [72]:
cols_history = ['version', 'timestamp', 'operation']
delta_object.history().select(cols_history).show(truncate=False)

+-------+-----------------------+---------+
|version|timestamp              |operation|
+-------+-----------------------+---------+
|2      |2022-07-11 13:29:16.574|MERGE    |
|1      |2022-07-11 13:29:09.952|UPDATE   |
|0      |2022-07-11 13:29:02.304|WRITE    |
+-------+-----------------------+---------+



In [73]:
# time travel per version
time_travel_version_0 = (
    spark
        .read
        .format('delta')
        .option('versionAsOf', '0') # versionAsOf / timestampAsOf
        .load('../delta/bronze/user/')
)

time_travel_version_0.show()
time_travel_version_0.count()

+-------+--------------------+
|user_id|               email|
+-------+--------------------+
|   1703|daron.bailey@emai...|
|   3650|jonah.barrows@ema...|
|   8809|carla.hansen@emai...|
|   4606|tomas.ledner@emai...|
|      1|alyse.ortiz@email...|
|   9245|russell.kulas@ema...|
|   3425|armida.lehner@ema...|
|   4264|tad.sanford@email...|
|   1668|rosia.jones@email...|
|    343|candy.conroy@emai...|
|   7393|dulcie.gottlieb@e...|
|   3909|rodrigo.reynolds@...|
|   9952|jenna.bode@email.com|
|   2364|dan.herman@email.com|
|   1611|stanley.witting@e...|
|   1723|clarinda.kilback@...|
|   7032|charley.carroll@e...|
|    549|cameron.harris@em...|
|   4161|reyes.stracke@ema...|
|    503|jolynn.schulist@e...|
+-------+--------------------+
only showing top 20 rows



600

In [74]:
# verify changes beetween to versions delta
df.exceptAll(delta_object.toDF()).show(truncate=False)

+-------+-----------------------+
|user_id|email                  |
+-------+-----------------------+
|3650   |jonah.barrows@email.com|
|1703   |daron.bailey@email.com |
|1805   |renee.hahn@email.com   |
+-------+-----------------------+



In [75]:
# read raw data
df = spark.read.format('json').load('../data/')

# insert new columns
columns = ['user_id', 'email', 'gender']
df = df.select(columns)
df.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- email: string (nullable = true)
 |-- gender: string (nullable = true)



In [76]:
# evolution schema
df.write.format('delta').mode('overwrite').option('mergeSchema', True).save('../delta/bronze/user') # mergeSchema / overwriteSchema

In [77]:
delta_object = DeltaTable.forPath(spark, '../delta/bronze/user/')
delta_object.toDF().show()

+-------+--------------------+-----------+
|user_id|               email|     gender|
+-------+--------------------+-----------+
|   3395|marcos.collier@em...|   Bigender|
|   1556|elina.hills@email...| Non-binary|
|   1879|enedina.schroeder...|     Female|
|   7805|colin.ryan@email.com|       Male|
|   3982|dallas.boyle@emai...| Polygender|
|   7274|grover.towne@emai...|    Agender|
|   3184|dexter.schmitt@em...|Genderfluid|
|    550|novella.weber@ema...|Genderqueer|
|   8365|lesley.mccullough...|       Male|
|   4942|marti.marks@email...|   Bigender|
|   8327|shawnna.keebler@e...|Genderfluid|
|   9464|guillermo.beahan@...|   Bigender|
|   4123|sid.bechtelar@ema...| Non-binary|
|   2281|merrill.upton@ema...|     Female|
|   6998|felipe.ward@email...|    Agender|
|   7440|willie.walsh@emai...|       Male|
|   8397|jae.krajcik@email...| Polygender|
|   9437|wilfredo.bailey@e...| Polygender|
|   3148|josefa.marvin@ema...|     Female|
|   2535|loyd.hintz@email.com| Polygender|
+-------+--

In [78]:
# file that will map the files inserted after the last write operation
delta_object.generate('symlink_format_manifest')

In [79]:
# write files in parquet format
df.coalesce(1).write.format('parquet').mode('overwrite').save('../parquet/user')

In [80]:
# converts the file type from parquet to delta, note that the _delta_log directory was created
delta_object.convertToDelta(spark, 'parquet.`../parquet/user/`')
!ls ../parquet/user/

_SUCCESS
_delta_log
part-00000-d2e397b4-6f5e-4ed6-88a2-d558a9031f24-c000.snappy.parquet
